### 1. 프로젝트 생성

In [6]:
!scrapy startproject naver_movie

New Scrapy project 'naver_movie', using template directory 'c:\users\samsung\anaconda\lib\site-packages\scrapy\templates\project', created in:
    C:\Users\samsung\Desktop\fastcampus\machine_learning_pj\naver_movie

You can start your first spider with:
    cd naver_movie
    scrapy genspider example example.com


### 2. xpath확인

In [1]:
import requests
import scrapy
from scrapy.http import TextResponse

In [14]:
genre = 10
url = 'https://movie.naver.com/movie/sdb/browsing/bmovie.nhn?genre={}'.format(genre)
req = requests.get(url)
response = TextResponse(req.url, body=req.text, encoding="utf-8")
response

<200 https://movie.naver.com/movie/sdb/browsing/bmovie.nhn?genre=10>

In [15]:
https://movie.naver.com/movie/sdb/browsing/bmovie.nhn?genre=2

SyntaxError: invalid syntax (<ipython-input-15-eb5365721d35>, line 1)

In [16]:
links = response.xpath('//*[@id="old_content"]/ul/li/a/@href').extract()
len(links)

20

In [17]:
link = response.urljoin(links[0])
link

'https://movie.naver.com/movie/bi/mi/basic.nhn?code=65348'

In [18]:
# 상세 데이터 수집
req = requests.get(link)
response = TextResponse(req.url, body=req.text, encoding="utf-8")

In [19]:
title = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/h3/a[1]/text()').extract()
title

['0.08']

In [20]:
story = response.xpath('//*[@id="content"]/div[1]/div[4]/div[1]/div/div/p/text()').extract()
story = " ".join(story).replace("\r \xa0", "")
story

'이 다큐멘터리는 스페인에 사는 빅토르라는 열 두 살 소년의 아름다운 초상이다. 빅토르도 그 또래 남자아이들처럼 마시멜로우 먹는 것을 좋아하고 해변가에서 뛰놀며 축구 하는 것을 즐긴다. 빅토르에게 한가지 다른 점이 있다면 그의 시력이 거의 장님에 가까운 0.08%란 점이다.'

### 3. items.py 작성

In [103]:
%%writefile naver_movie/naver_movie/items.py
import scrapy


class NaverMovieItem(scrapy.Item):
    # define the fields for your item here like:
    title = scrapy.Field()
    story = scrapy.Field()

Overwriting naver_movie/naver_movie/items.py


### 4. spider 작성

In [8]:
%%writefile naver_movie/naver_movie/spiders/spiders.py
import scrapy
from naver_movie.items import NaverMovieItem


class MovieSpider(scrapy.Spider):
    name = "NaverMovie"
    allow_domain = ["https://movie.naver.com"]
    start_urls = ["https://movie.naver.com/movie/sdb/browsing/bmovie.nhn?genre=21"]
    
    # 페이지별 링크 만들기
    def parse(self, response):
        last_page = 137
        for page in range(last_page + 1):
            page += 1
            link = response.url + "&page={}".format(page)
            yield scrapy.Request(link, callback=self.parse_page)
    
    # 각 영화별 상세페이지
    def parse_page(self, response):
        links = response.xpath('//*[@id="old_content"]/ul/li/a/@href').extract()
        for link in links:
            link = response.urljoin(link)
            yield scrapy.Request(link, callback=self.parse_page_contents)
    
    # 상세페이지에서 컨텐츠 가져오기 
    def parse_page_contents(self, response):
        item = NaverMovieItem()
        item["title"] = response.xpath('//*[@id="content"]/div[1]/div[2]/div[1]/h3/a[1]/text()').extract()[0]
        try:
            story = response.xpath('//*[@id="content"]/div[1]/div[4]/div[1]/div/div/p/text()').extract()
            story = " ".join(story).replace("\r \xa0", "")
            item["story"] = story
        except:
            item["story"] = None
            
        yield item

Overwriting naver_movie/naver_movie/spiders/spiders.py


### 5. Settings.py 파일 변경

In [33]:
# robots.txt 무시하기
!sed -i 's/ROBOTSTXT_OBEY = True/ROBOTSTXT_OBEY = False/' naver_movie/naver_movie/settings.py

### 6. Scrapy 실행

In [9]:
%%writefile run.sh
cd naver_movie
scrapy crawl NaverMovie -o naver_movie_erotic.csv

Overwriting run.sh


In [36]:
!chmod +x run.sh

### 7. DataFrame으로 불러오기

In [10]:
df = pd.read_csv("naver_movie/naver_movie_erotic.csv")
df = df[["title", "story"]]
len(df)

2206

In [12]:
df.to_csv("naver_movie/naver_movie_erotic.csv")

In [22]:
df = pd.read_csv("naver_movie/naver_movie_documentary.csv")
df[100:120]

,Unnamed: 0,title,story,genre
100,100,가린차 - 밀림의 영웅,NaN,다큐멘터리
101,101,가리봉,서울특별시 구로구 가리봉동은 서울 속의 타지이다. 가리봉 거리를 가득 채운 중국어 ...,다큐멘터리
102,102,초이스 :: 강길,아직 공사 이전의 아름다운 강을 다녀온 순례길에서 만난 주민과 순례객들의 인터뷰와 ...,다큐멘터리
103,103,가르보,NaN,다큐멘터리
104,104,가무샤라,NaN,다큐멘터리
105,105,가루다 파워: 인도네시아 액션영화의 힘,"1920-30년대 영화산업의 생성기에서 70년대 전성기, 90년대 초중반의 쇠퇴기,...",다큐멘터리
106,106,"가로막힌 자유, 집회",노벨상을 수상한 대통령이 집권하는 나라에서 인권 후진국이라는 오명을 벗지 못하고 있...,다큐멘터리
107,107,가라파,UN 통계에 따르면 전체 지구 인구 중 9억 2천만 명 가량이 기아에 허덕이고 있다...,다큐멘터리
108,108,가라지 피플,NaN,다큐멘터리
109,109,가디언즈 오브 디 어스,NaN,다큐멘터리
